# OSO

In [2]:
import polars as pl
from google.cloud import bigquery

client = bigquery.Client()

In [4]:
def bq_query(query: str) -> pl.DataFrame:
    query_job = client.query(query)
    rows = query_job.result()
    return pl.from_arrow(rows.to_arrow())  # type: ignore


In [6]:
query = """
with funding as (
    select
        date_trunc(time, quarter) as quarter,
        from_project_name as funder,
        grant_pool_name,
        to_project_name as project,
        to_project_id,
        cast(sum(amount) as int) as total_funding_usd
    from `opensource-observer.oso.oss_funding_v0`
    group by 1, 2, 3, 4, 5
),
repos as (
    select
        project_id,
        max_by(artifact_url, star_count) as project_repo
    from `opensource-observer.oso.repositories_v0`
    where artifact_url in ('https://github.com/libp2p/go-nat')
    group by project_id
)
select
    repos.project_repo,
    funding.* except (to_project_id),
from funding
join repos
    on funding.to_project_id = repos.project_id
"""

bq_query(q)


project_repo,quarter,funder,grant_pool_name,project,total_funding_usd
str,"datetime[μs, UTC]",str,str,str,i64
"""https://github.com/libp2p/go-n…",2024-01-01 00:00:00 UTC,"""optimism""","""retropgf3""","""libp2p""",1043482
"""https://github.com/libp2p/go-n…",2024-10-01 00:00:00 UTC,"""optimism""","""retrofunding5""","""libp2p""",652907


In [3]:
QUERY = """
select
  artifact_url as target,
  is_fork,
  star_count,
  fork_count,
  created_at,
  updated_at
from `opensource-observer.oso.repositories_v0`
where artifact_url in ('https://github.com/libp2p/go-libp2p')
"""

query_job = client.query(QUERY)
rows = query_job.result()

oso_df = pl.from_arrow(rows.to_arrow())

print(f"Rows: {oso_df.shape[0]}")

oso_df.head()

Rows: 1


target,is_fork,star_count,fork_count,created_at,updated_at
str,bool,i64,i64,"datetime[μs, UTC]","datetime[μs, UTC]"
"""https://github.com/libp2p/go-l…",false,6145,1095,2015-09-30 23:24:32 UTC,2024-12-26 15:37:42 UTC
